## Brazilian E-Commerce Data

### An analysis of E-Commerce data from Olist

the data can be found on Kaggle at [this link](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

In [9]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import seaborn as sns

In [10]:
path = "../brazil/"
files = glob.glob(path + "/*.csv")

files

['../brazil/olist_sellers_dataset.csv',
 '../brazil/product_category_name_translation.csv',
 '../brazil/olist_orders_dataset.csv',
 '../brazil/olist_order_items_dataset.csv',
 '../brazil/olist_customers_dataset.csv',
 '../brazil/olist_geolocation_dataset.csv',
 '../brazil/olist_order_payments_dataset.csv',
 '../brazil/olist_order_reviews_dataset.csv',
 '../brazil/olist_products_dataset.csv']

In [11]:
li = []

li = [pd.read_csv(filename, index_col=None, header=0) for filename in files]
df = pd.concat(li, axis=0, ignore_index=True)
df

,seller_id,seller_zip_code_prefix,seller_city,seller_state,product_category_name,product_category_name_english,order_id,customer_id,order_status,order_purchase_timestamp,...,review_comment_message,review_creation_date,review_answer_timestamp,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,3442f8959a84dea7ee197c632cb2df15,13023.0,campinas,SP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844.0,mogi guacu,SP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031.0,rio de janeiro,RJ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195.0,sao paulo,SP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51a04a8a6bdcb23deccc82b0b80742cf,12914.0,braganca paulista,SP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550917,NaN,NaN,NaN,NaN,moveis_decoracao,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
1550918,NaN,NaN,NaN,NaN,construcao_ferramentas_iluminacao,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
1550919,NaN,NaN,NaN,NaN,cama_mesa_banho,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
1550920,NaN,NaN,NaN,NaN,informatica_acessorios,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60.0,156.0,2.0,700.0,31.0,13.0,20.0
